<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_geracao_distribuida_bruto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Alunos: Nomes, Nomes**

Professores: Douglas Ribeiro e Franciane Rodrigues

## **Pergunta de negócio**: Análise exploratória de dados com a base de dados do ___ e indique possíveis oportunidades e/ou ameaças para o empreendimento.

---



# ETL
Extração, transformação, carregamento dos dados



## Preparação de Ambiente
Instalações e importações das bibliotecas necessárias para o processo de ETL.

In [ ]:
# instalaçao de biblioteca para interagir com google cloud
!pip install gcsfs -q

In [ ]:
# instalaçao de biblioteca para validar dados com pandera
!pip install pandera -q

In [ ]:
# Importando Bibliotecas
import os
import pandas as pd
import numpy as np
import pandera as pa
import re

from google.cloud import storage
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuração da quantidade de colunas para aparecer em um DataFrame
pd.set_option('display.max_columns',100)

### Compartilhamento Chave GDrive

In [ ]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada
target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


## Extração
A primeira etapa da ETL é a extração dos dados de sua fonte original. Dependendo do tipo de dados e da fonte, você pode precisar de diferentes ferramentas e técnicas para extrair os dados.

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket('projeto-final-ad2-e8') # nome do bucket

In [ ]:
# Caminhodo arquivo
path = "gs://projeto-final-ad2-e8/dados/pre-analise/geracao_distribuida_pre.csv" # gsutil

*Read* Pandas

In [ ]:
# Leitura dataframe pandas
read = pd.read_csv(path,
                         sep= ',',
                         encoding = 'utf-8')

In [ ]:
# exibindo primeiras linhas
read.head(10)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,2023-07-31,07/2023,8.336783e+12,CELESC-DIS,CELESC DISTRIBUI�ÃO S.A,5,Poder Público,11,B3,42.0,SC,4204.0,Sul,4210852.0,Mirim Doce,00000000,PJ,95952289000155,MIRIM DOCE CAMARA DE VEREADORES,GD.SC.000.188.684,2020-01-20,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,500,NaN,NaN,NaN,NaN,NaN
1,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,01109010,PJ,10882594001722,"INSTITUTO FEDERAL DE EDUCA�ÃO, CIÊNCIA E TECN.",GD.SP.000.027.612,2017-07-12,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6996,NaN,NaN,NaN,NaN,NaN
2,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,04115***,PF,***.015.418-**,***,GD.SP.000.081.118,2019-05-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
3,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3510609.0,Carapicuíba,06364***,PF,***.517.398-**,***,GD.SP.000.210.094,2020-03-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
4,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3505708.0,Barueri,06462080,PJ,46523015000135,MUNICIPIO DE BARUERI,GD.SP.000.863.525,2021-04-02,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,2300,-2351.0,-4688.0,NaN,NaN,NaN
5,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,11,B3,35.0,SP,3515.0,Sudeste,3518800.0,Guarulhos,07160640,PJ,46319000000150,MUNICIPIO DE GUARULHOS,GD.SP.000.251.356,2020-06-09,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,284,-2346.0,-4650.0,NaN,NaN,NaN
6,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,11,B3,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,08265060,PJ,74118514000182,SAO PAULO SECRETARIA VERDE MEIO AMB,GD.SP.000.049.685,2018-12-11,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
7,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3552502.0,Suzano,08683000,PJ,43776517000180,CIA DE SANEAMENTO BASICO DO ESTADO DE S,GD.SP.001.908.917,2023-06-07,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Minigeracao,200000,74.0,-20.0,NaN,NaN,NaN
8,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3530607.0,Mogi das Cruzes,08750770,PJ,46523270000188,MUNICIPIO DE MOGI DAS CRUZES,GD.SP.000.351.911,2020-10-19,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6000,-2300.0,-4700.0,NaN,NaN,NaN
9,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,9,B1,35.0,SP,3513.0,Sudeste,3550704.0,São Sebastião,11619***,PF,***.090.248-**,***,GD.SP.000.314.370,2020-10-05,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,200,NaN,NaN,NaN,NaN,NaN


In [ ]:
# renomeação do dataframe lido
df = read

## Pré-análise

In [ ]:
# Visualização geral
display(df)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,2023-07-31,07/2023,8.336783e+12,CELESC-DIS,CELESC DISTRIBUI�ÃO S.A,5,Poder Público,11,B3,42.0,SC,4204.0,Sul,4210852.0,Mirim Doce,00000000,PJ,95952289000155,MIRIM DOCE CAMARA DE VEREADORES,GD.SC.000.188.684,2020-01-20,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,500,NaN,NaN,NaN,NaN,NaN
1,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,01109010,PJ,10882594001722,"INSTITUTO FEDERAL DE EDUCA�ÃO, CIÊNCIA E TECN.",GD.SP.000.027.612,2017-07-12,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6996,NaN,NaN,NaN,NaN,NaN
2,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,04115***,PF,***.015.418-**,***,GD.SP.000.081.118,2019-05-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
3,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3510609.0,Carapicuíba,06364***,PF,***.517.398-**,***,GD.SP.000.210.094,2020-03-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
4,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3505708.0,Barueri,06462080,PJ,46523015000135,MUNICIPIO DE BARUERI,GD.SP.000.863.525,2021-04-02,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,2300,-2351.0,-4688.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053216,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,9,B1,24.0,RN,2404.0,Nordeste,2403251.0,Parnamirim,59152***,PF,***.368.374-**,***,GD.RN.000.549.804,2021-07-31,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,737,NaN,NaN,NaN,NaN,NaN
2053217,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,9,B1,24.0,RN,2402.0,Nordeste,2402006.0,Caicó,59300***,PF,***.405.104-**,***,GD.RN.000.863.246,2022-02-25,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
2053218,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,10,B2,24.0,RN,2402.0,Nordeste,2407203.0,Macau,59500***,PF,***.771.634-**,***,GD.RN.000.349.903,2020-11-06,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,600,NaN,NaN,NaN,NaN,NaN
2053219,2023-07-31,07/2023,7.047251e+12,ENEL CE,COMPANHIA ENERGÉTICA DO CEARÁ,7,Ilumina�ão pública,11,B3,23.0,CE,2303.0,Nordeste,2304400.0,Fortaleza,60878085,PJ,23552466000103,RENOVADORA DE P SAO LUIS LTDA,GD.CE.000.166.397,2019-12-31,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,4460,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Visualização de forma aleatoria
df.sample(20)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
884202,2023-07-31,07/2023,1.377555e+12,CHESP,COMPANHIA HIDROELÉTRICA SÃO PATRÍCIO - CHESP,1,Residencial,9,B1,52.0,GO,5203.0,Centro Oeste,5221700.0,Uruana,76335***,PF,***.722.501-**,***,GD.GO.001.734.264,2023-03-18,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,999,NaN,NaN,NaN,NaN,NaN
1625867,2023-07-31,07/2023,7.047251e+12,ENEL CE,COMPANHIA ENERGÉTICA DO CEARÁ,1,Residencial,9,B1,23.0,CE,2301.0,Nordeste,2302602.0,Camocim,62400000,PJ,23301562000170,A P FRIOS,GD.CE.001.112.316,2022-07-04,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6000,-290.0,-4084.0,NaN,NaN,NaN
271055,2023-07-31,07/2023,1.543032e+12,Equatorial GO,EQUATORIAL GOIÁS DISTRIBUIDORA DE ENERGIA S/A,1,Residencial,9,B1,52.0,GO,5203.0,Centro Oeste,5215009.0,Nova Veneza,75470***,PF,***.087.091-**,***,GD.GO.001.292.995,2022-09-26,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,404,NaN,NaN,NaN,NaN,NaN
1590675,2023-07-31,07/2023,3.305020e+13,CPFL-PAULISTA,COMPANHIA PAULISTA DE FORÇA E LUZ,1,Residencial,9,B1,35.0,SP,3504.0,Sudeste,3522901.0,Itapuí,17230***,PF,***.365.508-**,***,GD.SP.000.912.868,2022-03-14,R,Caracterizada como Autoconsumo remoto,3,UFV,Radia�ão solar,Microgeracao,910,NaN,NaN,NaN,NaN,NaN
3624,2023-07-31,07/2023,3.305020e+13,CPFL-PAULISTA,COMPANHIA PAULISTA DE FORÇA E LUZ,1,Residencial,9,B1,35.0,SP,3507.0,Sudeste,3509502.0,Campinas,13098***,PF,***.854.508-**,***,GD.SP.000.692.428,2021-11-12,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,850,NaN,NaN,NaN,NaN,NaN
1325023,2023-07-31,07/2023,4.368898e+12,COPEL-DIS,COPEL DISTRIBUI�ÃO S.A.,3,Rural,10,B2,41.0,PR,4101.0,Sul,4112405.0,Japurá,87970***,PF,***.878.599-**,***,GD.PR.000.677.784,2020-05-18,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,600,NaN,NaN,NaN,NaN,NaN
1853970,2023-07-31,07/2023,3.467321e+12,EMT,Energisa Mato Grosso - Distribuidora de Energi...,3,Rural,7,A4,51.0,MT,5101.0,Centro Oeste,5105259.0,Lucas do Rio Verde,78455***,PF,***.102.020-**,***,GD.MT.000.493.754,2021-04-05,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Minigeracao,10000,NaN,NaN,NaN,NaN,NaN
565808,2023-07-31,07/2023,3.305020e+13,CPFL-PAULISTA,COMPANHIA PAULISTA DE FORÇA E LUZ,1,Residencial,9,B1,35.0,SP,3503.0,Sudeste,3502804.0,Araçatuba,16022***,PF,***.356.028-**,***,GD.SP.001.002.316,2022-05-19,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,460,NaN,NaN,NaN,NaN,NaN
885148,2023-07-31,07/2023,5.914650e+12,CERON,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S.A,1,Residencial,9,B1,11.0,RO,1102.0,Norte,1100122.0,Ji-Paraná,76913***,PF,***.560.281-**,***,GD.RO.000.892.125,2020-01-03,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,1000,NaN,NaN,NaN,NaN,NaN
267313,2023-07-31,07/2023,4.895728e+12,Equatorial PA,EQUATORIAL PARÁ DISTRIBUIDORA DE ENERGIA S.A.,1,Residencial,9,B1,15.0,PA,1502.0,Norte,1501808.0,Breves,68800***,PF,***.609.672-**,***,GD.PA.000.704.906,2021-09-29,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,450,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Tamanho total de (linhas , colunas)
df.shape

(2053221, 33)

In [ ]:
# Todos os tipos de dados presentes
df.dtypes

DatGeracaoConjuntoDados          object
AnmPeriodoReferencia             object
NumCNPJDistribuidora            float64
SigAgente                        object
NomAgente                        object
CodClasseConsumo                  int64
DscClasseConsumo                 object
CodSubGrupoTarifario              int64
DscSubGrupoTarifario             object
codUFibge                       float64
SigUF                            object
codRegiao                       float64
NomRegiao                        object
CodMunicipioIbge                float64
NomMunicipio                     object
CodCEP                           object
SigTipoConsumidor                object
NumCPFCNPJ                       object
NomeTitularEmpreendimento        object
CodEmpreendimento                object
DthAtualizaCadastralEmpreend     object
SigModalidadeEmpreendimento      object
DscModalidadeHabilitado          object
QtdUCRecebeCredito                int64
SigTipoGeracao                   object


In [ ]:

# Contagem de observações para cada coluna
df.count()

DatGeracaoConjuntoDados         2053221
AnmPeriodoReferencia            2053221
NumCNPJDistribuidora            2053081
SigAgente                       2053081
NomAgente                       2053081
CodClasseConsumo                2053221
DscClasseConsumo                2053221
CodSubGrupoTarifario            2053221
DscSubGrupoTarifario            2053221
codUFibge                       2049092
SigUF                           2053041
codRegiao                       2049092
NomRegiao                       2053221
CodMunicipioIbge                2053041
NomMunicipio                    2053041
CodCEP                          2053219
SigTipoConsumidor               2053219
NumCPFCNPJ                      2053219
NomeTitularEmpreendimento       2053212
CodEmpreendimento               2053220
DthAtualizaCadastralEmpreend    2053221
SigModalidadeEmpreendimento     2053221
DscModalidadeHabilitado         2053040
QtdUCRecebeCredito              2053221
SigTipoGeracao                  2053221


In [ ]:
# Informações detalhadas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2053221 entries, 0 to 2053220
Data columns (total 33 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   DatGeracaoConjuntoDados       object 
 1   AnmPeriodoReferencia          object 
 2   NumCNPJDistribuidora          float64
 3   SigAgente                     object 
 4   NomAgente                     object 
 5   CodClasseConsumo              int64  
 6   DscClasseConsumo              object 
 7   CodSubGrupoTarifario          int64  
 8   DscSubGrupoTarifario          object 
 9   codUFibge                     float64
 10  SigUF                         object 
 11  codRegiao                     float64
 12  NomRegiao                     object 
 13  CodMunicipioIbge              float64
 14  NomMunicipio                  object 
 15  CodCEP                        object 
 16  SigTipoConsumidor             object 
 17  NumCPFCNPJ                    object 
 18  NomeTitularEmpreendime

In [ ]:
# Backup local
dfbackup1 = df.copy()

## Transformation
Aqui os dados extraídos precisam ser transformados em um formato adequado para análise, e para isso deve seguir alguns passos, que podem precisar serem repetidos ao longo do processo de análise.

### Limpeza de Dados
Remover dados duplicados, corrigir erros de digitação, tratar dados inconsistentes etc.

### Subsituições

Função regex_replace

In [ ]:
# função para substituição de caractere
def replace_single_char(text, target_char, replacement_char):
'''Essa função retornará caractere substituído'''
    return re.sub(target_char, replacement_char, text)

Substituição de caracter especial

In [ ]:
# definição dos caracteres
target_char = '�'
replacement_char = 'c'

# chamando a função
df = df.applymap(lambda x: replace_single_char(str(x), target_char, replacement_char))

Lower

In [ ]:
# padronizando registros em lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

Backup

In [ ]:
dfbackup2 = df.copy()

Substituição de caracter com expressão regular

In [ ]:
#df = df[~df.apply(lambda row: any('*' in str(cell) for cell in row), axis=1)]

target_char = '*'
replacement_char = '0'

df = df.applymap(lambda x: replace_single_char(str(x), re.escape(target_char), replacement_char))

In [ ]:
dfbackup3 = df.copy()

In [ ]:
# Verificar se há valores duplicados
df.duplicated().sum()

0

In [ ]:
# Verificar se há valores nulos/ausentes
df.isnull().sum()

DatGeracaoConjuntoDados         0
AnmPeriodoReferencia            0
NumCNPJDistribuidora            0
SigAgente                       0
NomAgente                       0
CodClasseConsumo                0
DscClasseConsumo                0
CodSubGrupoTarifario            0
DscSubGrupoTarifario            0
codUFibge                       0
SigUF                           0
codRegiao                       0
NomRegiao                       0
CodMunicipioIbge                0
NomMunicipio                    0
CodCEP                          0
SigTipoConsumidor               0
NumCPFCNPJ                      0
NomeTitularEmpreendimento       0
CodEmpreendimento               0
DthAtualizaCadastralEmpreend    0
SigModalidadeEmpreendimento     0
DscModalidadeHabilitado         0
QtdUCRecebeCredito              0
SigTipoGeracao                  0
DscFonteGeracao                 0
DscPorte                        0
MdaPotenciaInstaladaKW          0
NumCoordNEmpreendimento         0
NumCoordEEmpre

In [ ]:
# Verificar se há valores nulos/ausentes
df.isna().sum()

dataPesquisa                 0
periodoPesquisa              0
idDistribuidora              0
agenteAbreviado              0
agenteNome                   0
classeConsumo                0
nomeClasseConsumo            0
subgrupoTarifa               0
nomeSubgrupoTarifa           0
ufIbge                       0
uf                           0
regiaoSigla                  0
regiaoNome                   0
municipioIbge                0
municipio                    0
CEP                          0
tipoConsumidor               0
idConsumidor                 0
nomeTitularEmpreendimento    0
idEmpreendimento             0
cadastroEmpreendimento       0
tipoEmpreendimento           0
tipoHabilitado               0
creditoReceber               0
tipoGeracao                  0
fonteGeracao                 0
porte                        0
potenciaInstalada(KW)        0
coordNEmp                    0
coordEEmp                    0
subEstacao                   0
coordESub                    0
coordNSu

In [ ]:
# Fazendo um relatório para mostrar os valores únicos de cada atributo.
# O for vai garantir que a atividade seja feita automaticamentepara todos os atributos

for coluna in df:
    valor = (df[coluna].unique())
    print("-"*70)
    msg = f"Valores únicos para o atributo {coluna}:"
    print(f"{msg:^70}")
    print("-"*70)
    print(f"\n{valor}\n")
    print("-"*70)


----------------------------------------------------------------------
       Valores únicos para o atributo DatGeracaoConjuntoDados:        
----------------------------------------------------------------------

['2023-07-31']

----------------------------------------------------------------------
----------------------------------------------------------------------
         Valores únicos para o atributo AnmPeriodoReferencia:         
----------------------------------------------------------------------

['07/2023']

----------------------------------------------------------------------
----------------------------------------------------------------------
         Valores únicos para o atributo NumCNPJDistribuidora:         
----------------------------------------------------------------------

['8336783000190.0' '61695227000193.0' '2302100000106.0' '2328280000197.0'
 '33050196000188.0' '53859112000169.0' '4172213000151.0' '7282377000120.0'
 '60444437000146.0' '33050071000158.0'

In [ ]:
# Verificando os valores presentes na coluna para identificar inconsistencias
for coluna in df:
    valor = (df[coluna].groupby(df[coluna]).count())
    print("-"*70)
    msg = f"Contando valores {coluna}:"
    print(f"{msg:^70}")
    print("-"*70)
    print(f"\n{valor}\n")
    print("-"*70)

----------------------------------------------------------------------
              Contando valores DatGeracaoConjuntoDados:               
----------------------------------------------------------------------

DatGeracaoConjuntoDados
2023-07-31    2053221
Name: DatGeracaoConjuntoDados, dtype: int64

----------------------------------------------------------------------
----------------------------------------------------------------------
                Contando valores AnmPeriodoReferencia:                
----------------------------------------------------------------------

AnmPeriodoReferencia
07/2023    2053221
Name: AnmPeriodoReferencia, dtype: int64

----------------------------------------------------------------------
----------------------------------------------------------------------
                Contando valores NumCNPJDistribuidora:                
----------------------------------------------------------------------

NumCNPJDistribuidora
10532365000110.0      

In [ ]:
df['NumCoordNSub'].sample(5)

1655123    nan
224114     nan
896187     nan
127419     nan
1492780    nan
Name: NumCoordNSub, dtype: object

In [ ]:
# Verificando os valores presentes na coluna para identificar inconsistencias
for coluna in df:
  df[coluna] = df[coluna].replace('nan', '0')

In [ ]:
dfbackup4 = df.copy()

### Normalização de Dados
Colocar os dados em um formato padronizado para facilitar a análise.

Tipagem

In [ ]:
df.sample(3)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
1908874,2023-07-31,07/2023,15139629000194.0,coelba,companhia de eletricidade do estado da bahia c...,1,residencial,9,b1,29.0,ba,2903.0,nordeste,2910800.0,feira de santana,44059000,pf,000.344.355-00,000,gd.ba.000.901.782,2022-01-11,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,400,0,0,0,0,0
274867,2023-07-31,07/2023,25086034000171.0,eto,energisa tocantins distribuidora de energia s.a.,1,residencial,9,b1,17.0,to,1701.0,norte,1713205.0,miracema do tocantins,77650000,pf,000.241.171-00,000,gd.to.001.110.561,2022-06-13,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,600,0,0,0,0,0
433161,2023-07-31,07/2023,3467321000199.0,emt,energisa mato grosso - distribuidora de energi...,1,residencial,9,b1,51.0,mt,5104.0,centro oeste,5103403.0,cuiabá,78000000,pj,28884824000190,jacinto & cia ltda,gd.mt.000.557.500,2021-07-09,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,600,-1529.0,-5626.0,0,0,0


In [ ]:
# Realizando a tipagem das variaveis
'''df.coluna1 = df.coluna1.astype(int)'''

df.CodClasseConsumo = df.CodClasseConsumo.astype(int)
df.CodSubGrupoTarifario = df.CodSubGrupoTarifario.astype(int)
df.codUFibge = df.codUFibge.astype(float)
df.codRegiao = df.codRegiao.astype(float)
df.CodMunicipioIbge = df.CodMunicipioIbge.astype(float)
df.DthAtualizaCadastralEmpreend = df.DthAtualizaCadastralEmpreend.astype('datetime64[ns]')
df.QtdUCRecebeCredito = df.QtdUCRecebeCredito.astype(int)
df.MdaPotenciaInstaladaKW = df.MdaPotenciaInstaladaKW.astype(int)
df.NumCoordNEmpreendimento = df.NumCoordNEmpreendimento.astype(float)
df.NumCoordESub = df.NumCoordESub.astype(float)
df.NumCoordNSub = df.NumCoordNSub.astype(float)

In [ ]:
df['anoCadastral'] = df.DthAtualizaCadastralEmpreend.dt.year

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2053221 entries, 0 to 2053220
Data columns (total 34 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   DatGeracaoConjuntoDados       object        
 1   AnmPeriodoReferencia          object        
 2   NumCNPJDistribuidora          object        
 3   SigAgente                     object        
 4   NomAgente                     object        
 5   CodClasseConsumo              int64         
 6   DscClasseConsumo              object        
 7   CodSubGrupoTarifario          int64         
 8   DscSubGrupoTarifario          object        
 9   codUFibge                     float64       
 10  SigUF                         object        
 11  codRegiao                     float64       
 12  NomRegiao                     object        
 13  CodMunicipioIbge              float64       
 14  NomMunicipio                  object        
 15  CodCEP                        ob

Tradução | Ajustes nomes

In [ ]:
colunas_atuais = df.columns
colunas_atuais

Index(['DatGeracaoConjuntoDados', 'AnmPeriodoReferencia',
       'NumCNPJDistribuidora', 'SigAgente', 'NomAgente', 'CodClasseConsumo',
       'DscClasseConsumo', 'CodSubGrupoTarifario', 'DscSubGrupoTarifario',
       'codUFibge', 'SigUF', 'codRegiao', 'NomRegiao', 'CodMunicipioIbge',
       'NomMunicipio', 'CodCEP', 'SigTipoConsumidor', 'NumCPFCNPJ',
       'NomeTitularEmpreendimento', 'CodEmpreendimento',
       'DthAtualizaCadastralEmpreend', 'SigModalidadeEmpreendimento',
       'DscModalidadeHabilitado', 'QtdUCRecebeCredito', 'SigTipoGeracao',
       'DscFonteGeracao', 'DscPorte', 'MdaPotenciaInstaladaKW',
       'NumCoordNEmpreendimento', 'NumCoordEEmpreendimento', 'NomSubEstacao',
       'NumCoordESub', 'NumCoordNSub', 'anoCadastral'],
      dtype='object')

In [ ]:
subs_colunas = ['dataPesquisa', 'periodoPesquisa',
       'idDistribuidora', 'agenteAbreviado', 'agenteNome', 'classeConsumo',
       'nomeClasseConsumo', 'subgrupoTarifa', 'nomeSubgrupoTarifa',
       'ufIbge', 'uf', 'regiaoSigla', 'regiaoNome', 'municipioIbge',
       'municipio', 'CEP', 'tipoConsumidor', 'idConsumidor',
       'nomeTitularEmpreendimento', 'idEmpreendimento',
       'cadastroEmpreendimento', 'tipoEmpreendimento',
       'tipoHabilitado', 'creditoReceber', 'tipoGeracao',
       'fonteGeracao', 'porte', 'potenciaInstalada(KW)',
       'coordNEmp', 'coordEEmp', 'subEstacao',
       'coordESub', 'coordNSub', 'anoCadastral']

In [ ]:
df.columns = subs_colunas

In [ ]:
df.columns

Index(['dataPesquisa', 'periodoPesquisa', 'idDistribuidora', 'agenteAbreviado',
       'agenteNome', 'classeConsumo', 'nomeClasseConsumo', 'subgrupoTarifa',
       'nomeSubgrupoTarifa', 'ufIbge', 'uf', 'regiaoSigla', 'regiaoNome',
       'municipioIbge', 'municipio', 'CEP', 'tipoConsumidor', 'idConsumidor',
       'nomeTitularEmpreendimento', 'idEmpreendimento',
       'cadastroEmpreendimento', 'tipoEmpreendimento', 'tipoHabilitado',
       'creditoReceber', 'tipoGeracao', 'fonteGeracao', 'porte',
       'potenciaInstalada(KW)', 'coordNEmp', 'coordEEmp', 'subEstacao',
       'coordESub', 'coordNSub', 'anoCadastral'],
      dtype='object')

In [ ]:
df.dtypes

dataPesquisa                         object
periodoPesquisa                      object
idDistribuidora                      object
agenteAbreviado                      object
agenteNome                           object
classeConsumo                         int64
nomeClasseConsumo                    object
subgrupoTarifa                        int64
nomeSubgrupoTarifa                   object
ufIbge                              float64
uf                                   object
regiaoSigla                         float64
regiaoNome                           object
municipioIbge                       float64
municipio                            object
CEP                                  object
tipoConsumidor                       object
idConsumidor                         object
nomeTitularEmpreendimento            object
idEmpreendimento                     object
cadastroEmpreendimento       datetime64[ns]
tipoEmpreendimento                   object
tipoHabilitado                  

# **Qualidade e integridade dos dados**

### **Validação da estrutura dos dados (schema)**

In [ ]:
# Definindo o esquema de validação e o examinando
schema = pa.DataFrameSchema({
    'dataPesquisa': pa.Column(pa.String),
    'periodoPesquisa': pa.Column(pa.String),
    'idDistribuidora': pa.Column(pa.String),
    'agenteAbreviado': pa.Column(pa.String),
    'agenteNome': pa.Column(pa.String),
    'classeConsumo': pa.Column(pa.Int),
    'nomeClasseConsumo': pa.Column(pa.String),
    'subgrupoTarifa': pa.Column(pa.Int),
    'nomeSubgrupoTarifa': pa.Column(pa.String),
    'ufIbge': pa.Column(pa.Float),
    'uf': pa.Column(pa.String),
    'regiaoSigla': pa.Column(pa.Float),
    'regiaoNome': pa.Column(pa.String),
    'municipioIbge': pa.Column(pa.Float),
    'municipio': pa.Column(pa.String),
    'CEP': pa.Column(pa.String),
    'tipoConsumidor': pa.Column(pa.String),
    'idConsumidor': pa.Column(pa.String),
    'nomeTitularEmpreendimento': pa.Column(pa.String),
    'idEmpreendimento': pa.Column(pa.String),
    'cadastroEmpreendimento': pa.Column(pa.Timestamp),
    'tipoEmpreendimento': pa.Column(pa.String),
    'tipoHabilitado': pa.Column(pa.String),
    'creditoReceber': pa.Column(pa.Int),
    'tipoGeracao': pa.Column(pa.String),
    'fonteGeracao': pa.Column(pa.String),
    'porte': pa.Column(pa.String),
    'potenciaInstalada(KW)': pa.Column(pa.Int),
    'coordNEmp': pa.Column(pa.Float),
    'coordEEmp': pa.Column(pa.String),
    'subEstacao': pa.Column(pa.String),
    'coordESub': pa.Column(pa.Float),
    'coordNSub': pa.Column(pa.Float),
    'anoCadastral': pa.Column(pa.Int)
})

schema.validate(df)

,dataPesquisa,periodoPesquisa,idDistribuidora,agenteAbreviado,agenteNome,classeConsumo,nomeClasseConsumo,subgrupoTarifa,nomeSubgrupoTarifa,ufIbge,uf,regiaoSigla,regiaoNome,municipioIbge,municipio,CEP,tipoConsumidor,idConsumidor,nomeTitularEmpreendimento,idEmpreendimento,cadastroEmpreendimento,tipoEmpreendimento,tipoHabilitado,creditoReceber,tipoGeracao,fonteGeracao,porte,potenciaInstalada(KW),coordNEmp,coordEEmp,subEstacao,coordESub,coordNSub,anoCadastral
0,2023-07-31,07/2023,8336783000190.0,celesc-dis,celesc distribuicão s.a,5,poder público,11,b3,42.0,sc,4204.0,sul,4210852.0,mirim doce,00000000,pj,95952289000155,mirim doce camara de vereadores,gd.sc.000.188.684,2020-01-20,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,500,0.0,0,0,0.0,0.0,2020
1,2023-07-31,07/2023,61695227000193.0,eletropaulo,eletropaulo metropolitana eletricidade de são ...,6,serviço público,7,a4,35.0,sp,3515.0,sudeste,3550308.0,são paulo,01109010,pj,10882594001722,"instituto federal de educacão, ciência e tecn.",gd.sp.000.027.612,2017-07-12,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,6996,0.0,0,0,0.0,0.0,2017
2,2023-07-31,07/2023,61695227000193.0,eletropaulo,eletropaulo metropolitana eletricidade de são ...,5,poder público,9,b1,35.0,sp,3515.0,sudeste,3550308.0,são paulo,04115000,pf,000.015.418-00,000,gd.sp.000.081.118,2019-05-30,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,300,0.0,0,0,0.0,0.0,2019
3,2023-07-31,07/2023,61695227000193.0,eletropaulo,eletropaulo metropolitana eletricidade de são ...,5,poder público,9,b1,35.0,sp,3515.0,sudeste,3510609.0,carapicuíba,06364000,pf,000.517.398-00,000,gd.sp.000.210.094,2020-03-30,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,300,0.0,0,0,0.0,0.0,2020
4,2023-07-31,07/2023,61695227000193.0,eletropaulo,eletropaulo metropolitana eletricidade de são ...,5,poder público,7,a4,35.0,sp,3515.0,sudeste,3505708.0,barueri,06462080,pj,46523015000135,municipio de barueri,gd.sp.000.863.525,2021-04-02,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,2300,-2351.0,-4688.0,0,0.0,0.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053216,2023-07-31,07/2023,8324196000181.0,cosern,companhia energética do rio grande do norte co...,7,iluminacão pública,9,b1,24.0,rn,2404.0,nordeste,2403251.0,parnamirim,59152000,pf,000.368.374-00,000,gd.rn.000.549.804,2021-07-31,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,737,0.0,0,0,0.0,0.0,2021
2053217,2023-07-31,07/2023,8324196000181.0,cosern,companhia energética do rio grande do norte co...,7,iluminacão pública,9,b1,24.0,rn,2402.0,nordeste,2402006.0,caicó,59300000,pf,000.405.104-00,000,gd.rn.000.863.246,2022-02-25,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,300,0.0,0,0,0.0,0.0,2022
2053218,2023-07-31,07/2023,8324196000181.0,cosern,companhia energética do rio grande do norte co...,7,iluminacão pública,10,b2,24.0,rn,2402.0,nordeste,2407203.0,macau,59500000,pf,000.771.634-00,000,gd.rn.000.349.903,2020-11-06,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,600,0.0,0,0,0.0,0.0,2020
2053219,2023-07-31,07/2023,7047251000170.0,enel ce,companhia energética do ceará,7,iluminacão pública,11,b3,23.0,ce,2303.0,nordeste,2304400.0,fortaleza,60878085,pj,23552466000103,renovadora de p sao luis ltda,gd.ce.000.166.397,2019-12-31,p,com microgeracao ou minigeracao distribuida,1,ufv,radiacão solar,microgeracao,4460,0.0,0,0,0.0,0.0,2019


## **Colunas Persistidas**

=============================================================
STRING
=============================================================


* dataPesquisa                         
* periodoPesquisa                      
* idDistribuidora                      
* agenteAbreviado                      
* agenteNome                           
* nomeSubgrupoTarifa                   
* nomeClasseConsumo                    
* uf                                   
* regiaoNome                           
* municipio                            
* CEP                                  
* tipoConsumidor                       
* idConsumidor                         
* nomeTitularEmpreendimento            
* idEmpreendimento                     
* tipoEmpreendimento                   
* tipoHabilitado                       
* tipoGeracao                          
* fonteGeracao                         
* porte                                
* coordEEmp                            
* subEstacao                           

=============================================================
INT
=============================================================

* classeConsumo                         
* subgrupoTarifa                        
* anoCadastral                          
* creditoReceber                        
* potenciaInstalada(KW)                 

=============================================================
FLOAT
=============================================================

* coordNEmp                           
* coordESub                           
* coordNSub                           
* ufIbge                              
* regiaoSigla                         
* municipioIbge                       

=============================================================
DATETIME
=============================================================
* cadastroEmpreendimento       

## Copia de segurança do tratamento - Backup

In [ ]:
# Backup para analise
df_tratado = df.copy()

## Load
Depois que os dados são extraídos e transformados adequadamente, eles estarão prontos para as análise, mas antes disso eles precisam ser carregados em um local de armazenamento adequado. Podendo ser um banco de dados SQL ou NoSQL, um sistema de armazenamento em nuvem, e para o caso de disponibilizar o projeto publicamente é ideal que ele seja colocar em uma pasta de datasets, diferenciando o arquivo bruto e o tratado.

In [ ]:
# Carregametno GCP
df_tratado.to_csv('gs://projeto-final-ad2-e8/dados/analise/Analise_geracao_distribuida.csv', index=False)